In [2]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pyranges as pr
import itertools
import sys
sys.path.insert(0, '/Genomics/kocherlab/bjarnold/STARRseq/code/notebooks')

import functions as fn
from collections import defaultdict
import os
from pyfaidx import Fasta
from Bio.Seq import Seq



In [3]:
species = "Nmel"
bioreps = ['F1', 'F2', 'F3']

chr1_test = False
qval_threshold = 1 # 1.3 corresponds to pvalue of 0.05, 1 corresponds to pvalue of 0.1
macs_folddiff_threshold = 2
min_interval_size = 200




In [4]:
base_dir = "/Users/bjarnold/Princeton_EEB/Kocher/STARRseq"
wd = f"{base_dir}/data/peak_calling_final/{species}"

# load MACS peak files
macs_peaks_files = fn.get_files(f'{wd}/MACS2', "*_peaks.narrowPeak")
print(macs_peaks_files)

#load ref genome
ref_genome = f'{base_dir}/data/alternate_references/renamed/{species}.fasta'

[]


In [5]:
macs_dfs = fn.load_peak_caller_results(macs_peaks_files, chr1_test) # list of macs dataframes

# FILTER PEAK FILES BY SIGNIFICANCE, EFFECT SIZE
print('MACS')
macs_dfs = fn.filter_by_sig_effect_size(macs_dfs, qval_threshold, macs_folddiff_threshold)

print([len(df) for df in macs_dfs])



MACS
before filtering: 9644
after filtering: 4305
before filtering: 13149
after filtering: 5501
before filtering: 12243
after filtering: 4904
Genrich
before filtering: 132922
after filtering: 46182
before filtering: 121542
after filtering: 29869
before filtering: 134246
after filtering: 40119
[4305, 5501, 4904]


In [6]:
# can't read with pandas because variable number of columns; some rows marked with an asterisk if the repeat overlaps with another higher-scoring repeat
# load by reading in all lines instead, then importing into table
# then use pyranges to merge overlapping intervals
repeat_pr = fn.load_repeat_modeler_intervals(repeat_file)

# open reference genome ref_genome, count the length of each chromosome, and store in a dictionary
fasta_sequences = Fasta(ref_genome)

scaff_lengths = defaultdict(int)
# Iterate over the sequences and print their names and lengths
for sequence_name in fasta_sequences.keys():
    scaff_lengths[sequence_name] = len(fasta_sequences[sequence_name])


FileNotFoundError: [Errno 2] No such file or directory: '/Genomics/kocherlab/bjarnold/STARRseq/data/peak_calling_snakemake_output/old/data/repeat_modeler_intervals/Nmel_repeat_elements.bed'

In [8]:
for m in macs_dfs:
    # convert peak/summit coordinate into an interval to overlap with pyranges
    m['peak_coord_1'] = m['peak_coord'] + 1

# gather relevant columns, put in new list of dataframes
macs_summit_dfs = [m[['Chromosome', 'peak_coord', 'peak_coord_1', 'signalValue']] for m in macs_dfs]

# rename columns to match pyranges format
for m in macs_summit_dfs:
    m.columns = ['Chromosome', 'Start', 'End', 'signalValue']

# convert df to pyranges object
macs_summit_prs = [pr.PyRanges(m) for m in macs_summit_dfs]



In [ ]:
kmer_df['sequence'] = pr.get_sequence(pr.PyRanges(kmer_df), ref_genome)
